# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成計數編碼以及搭配邏輯斯迴歸的預測 (In[4], Out[4], In[5], Out[5]) 
- 仿造範例, 完成雜湊編碼, 以及計數編碼+雜湊編碼 搭配邏輯斯迴歸的預測 (In[6], Out[6], In[7], Out[7]) 
- 試著回答上述執行結果的觀察

# 計數編碼(Counting Encoder) & 特徵雜湊(Feature Hashing)
- 計數編碼是計算類別在資料中的出現次數，當目標平均值與類別筆數呈正/負相關時，可以考慮使⽤用。也可以將筆數本⾝身當成特徵，例如 
    - 購物網站的消費金額預測，消費金額與消費筆數。
    - 自然語⾔言處理理時，字詞的計數編碼⼜又稱詞頻，本⾝身就是一個很重要的特徵
- 當相異異類別數量量相當⼤大時，其他編碼⽅方式效果更更差，可以考慮雜湊編碼以節省時間。
    - 特徵雜湊是⼀一種折衷⽅方案
    - 將類別由雜湊函數定應到⼀一組數字
    - 調整雜湊函數對應值的數量量
    - 在計算空間/時間與鑑別度間取折衷，也提⾼高了了訊息密度, 減少無⽤用的標籤

*註 : 雜湊編碼效果也不佳，這類問題更更好的解法是嵌入式編碼
(Embedding)，但是需要深度學習並有其前提，因此這裡暫時不排入課程

# 參考資料
- [Feature hashing (特徵哈希)](https://blog.csdn.net/laolu1573/article/details/79410187)
> 由下圖可以理解 : 雜湊編碼是比標籤編碼(上表)更緊密的編碼方式(下表)但要注意的是這樣的編碼:雖然在計算上比獨熱編碼省去很多時間，但是關鍵在雜湊後的特徵是否有意義。這邊有除了範例以外的細節講述，提供各位同學參考。![image.png](https://lh3.googleusercontent.com/ZPp6tAKa37OkxYRAfvHLxEM9O8N4DAbLji0-FLHNZIMPOdaoXMSau0nQe4NGPYaypVRT_ZcgMg8OuxkOrTcWzbCkVmuQjGpYTfRILPtLr1HLIaUQp3c_KqgXnXJ7EMlZNlvsaQ4DJ30?t=1557392456908)
- [基於sklearn的文本特徵抽取](https://www.jianshu.com/p/063840752151)
> 這裡講到的是 count vectorizer 與 tfidf vectorizer，是自然語言處理 (NLP) 時用的基礎技術之一，其中 count vectorizer 就是一種計數編碼的變形。雖然上述兩種編碼方式現階段暫時不用弄懂，但是我們可以藉此理解 : 計數編碼有其泛用性，甚至我們可以這樣理解 - **不需要局限於我們教會各位的編碼方式，只要在您的知識中有更適合的擷取特徵方式，並且能使用程式寫作出來的，建議不妨一試，就算不是泛用的編碼法，只要包含領域知識就可能有用。**

# 作業1
* 參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 目標均值編碼三種轉換後，  
與其他類別型欄位一起預估生存機率

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'data/part2/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Object Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 承上題，三者比較效果何者最好?
> 'Cabin'計數編碼 > 'Cabin'計數編碼 + Hash > 'Cabin' Hash > 標籤編碼。  
所以可以看出, 計數編碼是相對比較有效的特徵。  
特徵雜湊依照不同的Hash空間, 效果未必理想, 只是資訊比較密集, 計算上較快。  
混合使用也未必比只有計數編碼的效果好。

In [3]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
result_1 = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
print(result_1)
df_temp.head()

0.780004837244799


,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


### [pandas.DataFrame.agg(func, axis=0, *args, **kwargs)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html)
>Aggregate using one or more operations over the specified axis.

-  func : function, str, list or dict
Function to use for aggregating the data. If a function, must either work when passed a DataFrame or when passed to DataFrame.apply.

    Accepted combinations are:

    - function
    - string function name
    - list of functions and/or function names, e.g. [np.sum, 'mean']
    - dict of axis labels -> functions, function names or list of such.
- axis : {0 or ‘index’, 1 or ‘columns’}, default 0
    
    If 0 or ‘index’: apply function to each column. If 1 or ‘columns’: apply function to each row.

In [4]:
# 加上 'Cabin' 欄位的計數編碼
count_df = df.groupby(['Cabin'])['Name'].agg({'Cabin_Count':'size'}).reset_index()
df = pd.merge(df, count_df, on=['Cabin'], how='left')
count_df.sort_values(by=['Cabin_Count'], ascending=False).head(10)

,Cabin,Cabin_Count
185,None,1014
80,C23 C25 C27,6
184,G6,5
47,B57 B59 B63 B66,5
60,B96 B98,4
180,F2,4
117,D,4
79,C22 C26,4
181,F33,4
183,F4,4


In [5]:
df.head()

,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,1014
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,2
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1014
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,2
4,"Allen, Mr. William Henry",male,373450,None,S,1014


In [6]:
# 'Cabin'計數編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Cabin_Count'] = df['Cabin_Count']
train_X = df_temp[:train_num]
estimator = LogisticRegression()
result_2 = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
print(result_2)
df_temp.head()

0.7856230275549181


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count
0,155,1,720,185,3,1014
1,286,0,816,106,0,2
2,523,0,914,185,3,1014
3,422,0,65,70,3,2
4,22,1,649,185,3,1014


In [7]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
# 這邊的雜湊編碼, 是直接將 'Ticket' 的名稱放入雜湊函數的輸出數值, 為了要確定是緊密(dense)特徵, 因此除以10後看餘數
# 這邊的 10 是隨機選擇, 不一定要用 10, 同學可以自由選擇購小的數字試看看. 基本上效果都不會太好
df_temp['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 10)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
result_3 = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
print(result_3)
df_temp.head()


0.78112850367776


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash
0,155,1,720,185,3,9
1,286,0,816,106,0,8
2,523,0,914,185,3,9
3,422,0,65,70,3,0
4,22,1,649,185,3,9


In [8]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Cabin_Count'] = df['Cabin_Count']
df_temp['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 10)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
result_4 = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
print(result_4)
df_temp.head()

0.7822521701107211


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count,Cabin_Hash
0,155,1,720,185,3,1014,9
1,286,0,816,106,0,2,8
2,523,0,914,185,3,1014,9
3,422,0,65,70,3,2,0
4,22,1,649,185,3,1014,9


In [9]:
methods = ['LabelEncoder', 'LabelEncoder + Cabin CountingEncoder', 'LabelEncoder + Cabin Hashing',  'LabelEncoder + Cabin CountingEncoder + Cabin Hashing' ]
results = [result_1, result_2, result_3, result_4]
comparison = pd.DataFrame({
    'Encoder':methods,
    'Cross Val Score': results
})
print('Logistic Regrssion with Different Encoder Methods')
comparison.sort_values(by='Cross Val Score', ascending=False)

Logistic Regrssion with Different Encoder Methods


,Encoder,Cross Val Score
1,LabelEncoder + Cabin CountingEncoder,0.785623
3,LabelEncoder + Cabin CountingEncoder + Cabin H...,0.782252
2,LabelEncoder + Cabin Hashing,0.781129
0,LabelEncoder,0.780005


In [10]:
max_index = comparison['Cross Val Score'].argmax()
print('Max Score Encoder Method:', comparison['Encoder'][max_index])
print('Max Score:', comparison['Cross Val Score'][max_index])

Max Score Encoder Method: LabelEncoder + Cabin CountingEncoder
Max Score: 0.7856230275549181
